In [2]:
import docker
from IPython.display import display

In [ ]:
client = docker.from_env()
client.containers.run("ubuntu", "echo hello world")

In [ ]:
client.containers.run('alpine', 'echo hello world')

In [3]:
def run_container(image, command, **kwargs):
    try:
        container = client.containers.run(
        image=image,
        command=command,
        detach=True,
        **kwargs
        )
        for line in container.logs(stream=True, stdout=True, stderr=True):
            print(line.strip())

    except docker.errors.ContainerError as e:
        print(e.stderr.decode())
        raise e
    
    except docker.errors.ImageNotFound as e:
        print(e.stderr.decode())
        raise e
    
    except docker.errors.APIError as e:
        print(e.stderr.decode())
        raise e
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise e
    
    finally:
        print("Done")
        if 'container' in locals():
            container.stop()

In [4]:

container = client.containers.run(
    image='bfirsh/reticulate-splines',
    detach=True
)

for i, line in enumerate(container.logs(stream=True)):
    print(line.strip())

    if i > 10:
        container.stop()
        break

NameError: name 'client' is not defined

In [ ]:
container = client.containers.run(
    image='sum-app', 
    command='python sum.py 3 3',
    detach=True,
)
for line in container.logs(stream=True, stdout=True):
    print(line.strip())

In [1]:
import docker
from docker.errors import ImageNotFound, ContainerError, APIError

#### Example 1

In [5]:
def run_container(image, command, **kwargs):

    client = docker.from_env()

    try:
        container = client.containers.run(
        image=image,
        command=command,
        detach=True,
        **kwargs
        )
        for line in container.logs(stream=True, stdout=True, stderr=True):
            print(line.strip())

    except docker.errors.ContainerError as e:
        print(e)
        raise e
    
    except docker.errors.ImageNotFound as e:
        print(e)
        raise e
    
    except docker.errors.APIError as e:
        print(e)
        raise e
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise e
    
    finally:
        print("Done")
        if 'container' in locals():
            container.stop()

        # close client
        client.close()

In [7]:
run_container('sum-app', 'python sum.py --num1 3 --num2 3')

b'2023-12-20 06:13:06,747 - __main__ - INFO - Logger round 20'
b'2023-12-20 06:13:07,755 - __main__ - INFO - Logger round 19'
b'2023-12-20 06:13:08,765 - __main__ - INFO - Logger round 18'
b'2023-12-20 06:13:09,774 - __main__ - INFO - Logger round 17'
b'2023-12-20 06:13:10,782 - __main__ - INFO - Logger round 16'
b'2023-12-20 06:13:11,784 - __main__ - INFO - Logger round 15'
b'2023-12-20 06:13:12,800 - __main__ - INFO - Logger round 14'
b'2023-12-20 06:13:13,814 - __main__ - INFO - Logger round 13'
b'2023-12-20 06:13:14,821 - __main__ - INFO - Logger round 12'
b'2023-12-20 06:13:15,836 - __main__ - INFO - Logger round 11'
b'2023-12-20 06:13:16,843 - __main__ - INFO - Logger round 10'
b'2023-12-20 06:13:17,854 - __main__ - INFO - Logger round 9'
b'2023-12-20 06:13:18,861 - __main__ - INFO - Logger round 8'
b'2023-12-20 06:13:19,865 - __main__ - INFO - Logger round 7'
b'2023-12-20 06:13:20,873 - __main__ - INFO - Logger round 6'
b'2023-12-20 06:13:21,877 - __main__ - INFO - Logger round 

#### Example 2

In [ ]:

def run_container(image, command, **kwargs):
    client = docker.from_env()
    container = None
    try:
        container = client.containers.run(
            image=image,
            command=command,
            detach=True,
            **kwargs
        )
        
        # Using low-level API to attach to container streams
        low_level_api = docker.APIClient()
        output = low_level_api.attach(container.id, stream=True, logs=True, stdout=True, stderr=True)

        for chunk in output:
            print(chunk.decode('utf-8').strip())
            # Docker outputs in a multiplexed format: first byte for stdout/stderr, next 3 bytes for size
            stream_type = chunk[0]
            # 1 represents stdout, 2 represents stderr
            if stream_type == 1:
                print("STDOUT:", chunk[8:].decode('utf-8').strip())
            elif stream_type == 2:
                print("STDERR:", chunk[8:].decode('utf-8').strip())

    except ContainerError as e:
        print("Container error:", e.stderr.decode())
        raise e
    
    except ImageNotFound as e:
        print("Image not found:", e.stderr.decode())
        raise e

    except APIError as e:
        print("API error:", e.stderr.decode())
        raise e

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise e

    finally:
        print("Done")
        if container:
            container.stop()

        # close the low-level API client
        low_level_api.close()

        # close client
        client.close()



In [ ]:
run_container('sum-app', 'python sum.py 3 3')

In [8]:
import docker

client = docker.from_env()

# Replace with your specific details
image_name = "sum-app"
docker_command = "python sum.py 3 3"
# container_name = image_name
# ports = {"80/tcp": 8080}  # Optional: Map ports (container: host)
# environment = {"ENV_VAR": "value"}  # Optional: Environment variables

# Run the container
container = client.containers.run(image_name, 
                                  command=docker_command,
                                  # name=container_name,
                                #   ports=ports, 
                                #   environment=environment, 
                                  detach=True
            )

# Follow logs
for line in container.logs(stream=True, follow=True):
    print(line.strip())

b'Round 20'
b'Round 19'
b'Round 18'
b'Round 17'
b'Round 16'
b'Round 15'
b'Round 14'
b'Round 13'
b'Round 12'
b'Round 11'
b'Round 10'
b'Round 9'
b'Round 8'
b'Round 7'
b'Round 6'
b'Round 5'
b'Round 4'
b'Round 3'
b'Round 2'
b'Round 1'
b'Round 0'
b'The sum of 3.0 and 3.0 is 6.0'


In [9]:
import docker

In [14]:
client = docker.from_env()

def run_container(image, command, **kwargs):
    try:
        container = client.containers.run(
            image=image,
            command=command,
            detach=True,
            **kwargs
        )
        
        for line in container.logs(stream=True, follow=True):
            print(line.strip().decode("utf-8"))
        
    except (ContainerError, ImageNotFound, APIError) as e:
        print(e)
        raise e
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise e
    
    finally:
        try:
            container.remove()
        except Exception as e:
            print(f"An error occurred while removing container: {str(e)}")
            pass

In [15]:
run_container('sum-app', 'python -u sum.py 3 3')

Round 20
Round 19
Round 18
Round 17
Round 16
Round 15
Round 14
Round 13
Round 12
Round 11
Round 10
Round 9
Round 8
Round 7
Round 6
Round 5
Round 4
Round 3
Round 2
Round 1
Round 0
The sum of 3.0 and 3.0 is 6.0
Container status: created
